# Negative control datasets

## Imports

In [1]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML
import scipy
import torch
import pickle
import re
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
from inspect import signature

In [2]:
from scvi.models import AutoZIVAE
from scvi.inference import UnsupervisedTrainer
from datasets.negativecontrol import Svensson1NegativeControlDataset, Svensson2NegativeControlDataset,\
    KleinNegativeControlDataset, ZhengNegativeControlDataset
import torch
import pickle
import argparse
import re
import numpy as np
import time
from scvi.models.log_likelihood import compute_marginal_log_likelihood_scvi, compute_marginal_log_likelihood_autozi
import os
from autozi_simulate_tools import retrieve_rates_dropouts
from classification_metrics import *
from functools import partial

[2019-10-11 21:11:05,539] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/oscar/miniconda3/lib/python3.7/site-packages/scikit_learn-0.19.2-py3.7-linux-x86_64.egg/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
plt.switch_backend("TkAgg")
%matplotlib inline

## Train AutoZI on datasets

For each dataset under scrutiny, we retrieve the posterior parameters $\alpha^g, \beta^g$ of $q(\delta_g)$ for each gene $g$.

In [4]:
results_autozi_outputs = []
datasets_mapper = {
    'sven1-100rna': partial(Svensson1NegativeControlDataset, n_rna=100, threshold=None),
    'sven2-100rna': partial(Svensson2NegativeControlDataset, n_rna=100, threshold=None),
    'svenklein-100rna': partial(KleinNegativeControlDataset, n_rna=100, threshold=None),
    'svenzheng-0rna': partial(ZhengNegativeControlDataset, n_rna=0, threshold=None),
}

for dataset_name in datasets_mapper:
    data = datasets_mapper[dataset_name]()

    np.random.seed(int(time.time()))
    torch.manual_seed(int(time.time()))
    model = AutoZIVAE(n_input=data.nb_genes, alpha_prior=0.5, beta_prior=0.5,minimal_dropout=0.01)
    trainer = UnsupervisedTrainer(model, data)
    if 'sven1' in dataset_name or 'sven2' in dataset_name:
        trainer.train(n_epochs=600, lr=1e-3)
    else:
        trainer.train(n_epochs=600, lr=1e-2)
    outputs = trainer.model.get_alphas_betas(as_numpy=True)
    outputs['is_ercc'] = data.is_ercc
    outputs['dataset_name'] = dataset_name
    outputs['means_emp'] = data.X.mean(axis=0)
    results_autozi_outputs.append(outputs)

[2019-10-11 21:11:07,494] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-10-11 21:11:07,495] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


training: 100%|██████████| 600/600 [01:25<00:00,  7.67it/s]


[2019-10-11 21:12:36,279] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-10-11 21:12:36,280] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


training: 100%|██████████| 600/600 [01:25<00:00,  6.71it/s]


[2019-10-11 21:14:02,057] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-10-11 21:14:02,058] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


training: 100%|██████████| 600/600 [00:44<00:00, 12.03it/s]

[2019-10-11 21:14:46,712] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-10-11 21:14:46,713] INFO - scvi.dataset.dataset | Remapping labels to [0,N]



training: 100%|██████████| 600/600 [00:50<00:00, 13.29it/s]


## Compute metrics from AutoZI's outputs

For each dataset under scrutiny, from these $\alpha^g, \beta^g$, we estimate the ZI probabilities $q(\delta_g < 0.5)$ and classification metrics on the default decision rule $q(\delta_g < 0.5) > 0.5$ using tools from `classification_metrics.py`.

In [5]:
results_autozi_data_list = []

for outputs in results_autozi_outputs:
    
    is_ercc = outputs['is_ercc']
    means_emp = outputs["means_emp"]
    
    # Select spike-ins and control RNAs only with average expression above 1
    mask_means_emp = (means_emp > 1) 

    # The ground-truth is defined as NB for all genes for computational purposes.
    # However, the purpose of this study is to assess the relevance of this choice
    is_zinb_gt = np.zeros(is_ercc.shape).astype(bool)

    # Obtain classification results for each of ERCC spike-ins and control RNAs
    masks = {'ercc': (is_ercc & mask_means_emp), 'rna': ((~is_ercc) & (mask_means_emp))}

    metric_list_bernoulli = [
        ConfusionMatrixMetric(is_zinb_gt, masks=masks),
    ]

    model_score_evals = [
        AutoZIBernoulliThresholdEval('bernoullithreshold50', outputs, metric_list_bernoulli, threshold=0.50),
    ]

    results_autozi_data = {}
    for model_score_eval in model_score_evals:
        # Compute metrics for AutoZI on the dataset (here confusion matrix metrics)
        # Positives are ZI genes
        results_autozi_data.update(model_score_eval.compute_all_metrics())
        # Also directly add the scores used to compute the metrics
        results_autozi_data[model_score_eval.name] = model_score_eval.scores
        # Restrict the scores to ERCC spike-ins
        results_autozi_data[model_score_eval.name + '_ercc'] = model_score_eval.scores[is_ercc]
        # Restrict them to control RNAs
        results_autozi_data[model_score_eval.name + '_rna'] = model_score_eval.scores[~is_ercc]
        
        
        for key in ['means_emp','dataset_name']:
            results_autozi_data[key] = outputs.get(key,None)
            if key == 'means_emp':
                results_autozi_data[key + '_ercc'] = outputs.get(key, None)[is_ercc]
                results_autozi_data[key + '_rna'] = outputs.get(key, None)[~is_ercc]

    results_autozi_data_list.append(results_autozi_data)

results_autozi = pd.DataFrame(results_autozi_data_list)

/media/storage/Documents/2. Professionnel/UC Berkeley Internship 2019/scVI-C/autozi_reproducibility/classification_metrics.py:45: RuntimeWarning: Mean of empty slice.
  results['accuracy'] = (labels_predicted == labels_gt).mean()
/home/oscar/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
results_autozi = results_autozi.sort_values(by=['dataset_name']).set_index(['dataset_name'])

## Percentages of ZINB genes

ERCC

In [7]:
print("% of ZINB genes for ERCC spike-ins\n")
print(results_autozi.bernoullithreshold50_confusionmatrix_ercc_fp / results_autozi.bernoullithreshold50_confusionmatrix_ercc_total)
print('\nIn general : ', \
      results_autozi.bernoullithreshold50_confusionmatrix_ercc_fp.sum()\
      / results_autozi.bernoullithreshold50_confusionmatrix_ercc_total.sum())

% of ZINB genes for ERCC spike-ins

dataset_name
sven1-100rna        0.041667
sven2-100rna        0.083333
svenklein-100rna    0.000000
svenzheng-0rna      0.022727
dtype: float64

In general :  0.03333333333333333


RNA

In [8]:
print("% of ZINB genes for control RNAs\n")
print(results_autozi.bernoullithreshold50_confusionmatrix_rna_fp / results_autozi.bernoullithreshold50_confusionmatrix_rna_total)
print('\nIn general : ', \
      results_autozi.bernoullithreshold50_confusionmatrix_rna_fp.sum()\
      / results_autozi.bernoullithreshold50_confusionmatrix_rna_total.sum())

% of ZINB genes for control RNAs

dataset_name
sven1-100rna        0.06
sven2-100rna        0.09
svenklein-100rna    0.00
svenzheng-0rna       NaN
dtype: float64

In general :  0.05
